In [1]:
import os
import glob
import numpy as np
import bokeh.io
import bokeh.plotting
import bokeh.palettes
from bokeh.transform import jitter
import seaborn as sns
import matplotlib
from bokeh.models import HoverTool
from scipy import stats
import pandas as pd


from bokeh.layouts import row
bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
#Functions
def bootstrap_sampling(my_array, bootstrap_samples, bootstrap_replicates):
    bs_samples=np.zeros((bootstrap_replicates, bootstrap_samples))
    bs_rep=np.arange(bootstrap_replicates)
    for i in bs_rep:
        bs_samples[i, :]=np.random.choice(my_array, bootstrap_samples)
    return bs_samples

def bootstrap_stats(bs_samples):
    samples_shape=np.shape(bs_samples)
    bs_means=np.zeros((samples_shape[0], 1))
    bs_IC_means=np.zeros((1, 2))
    bs_medians=np.zeros((samples_shape[0], 1))
    bs_IC_medians=np.zeros((1, 2))
    for i in np.arange(samples_shape[0]):
        bs_means[i]=np.mean(bs_samples[i, :])
        bs_medians[i]=np.median(bs_samples[i, :])
    bs_IC_means[0, 0]=np.quantile(bs_means, .025)
    bs_IC_means[0, 1]=np.quantile(bs_means, .975)
    bs_IC_medians[0, 0]=np.quantile(bs_medians, .025)
    bs_IC_medians[0, 1]=np.quantile(bs_medians, .975)
    return np.mean(bs_means), np.median(bs_medians), bs_IC_means, bs_IC_medians

In [7]:
#set root folder
#path='E:\Sauvegardes PhD\Covid-19\Pili_and_PaQa_counts_data\Pili_fluo\mNG_pilH'
path = "C:/Users/tala/Desktop/git/PhD_codes/Mechanosensation/Python_code/Pole_analysis/combined_pilH/"
Combine_Data = True
os.chdir(path)
extension = 'csv'
list_csv = []
for root, dirs, files in os.walk(path, topdown=False):
    for name in files:
        if extension in name:
            list_csv.append(os.path.join(root, name))
#print(list_csv)
os.chdir("C:/users/tala/Desktop/git/PhD_codes/Mechanosensation/Python_code/Pole_analysis/")
new_dir = "combined_pilH\\"
saved_file = "PolesData_pilH_combined.csv"
if not os.path.exists(new_dir):
    os.mkdir(new_dir)
os.chdir(new_dir)
combined_csv = pd.concat([pd.read_csv(f) for f in list_csv ], sort=False)
combined_csv['Strain'] = 'nan'
combined_csv['Growth'] = 'nan'

raw_names=list(combined_csv.Label.unique())
#print(raw_names)

if not Combine_Data:
    for file in raw_names:
        #print(file)
        split_name=file.rsplit('_event',100)[0].rsplit('_',100)
        if 'Agarose' in split_name or 'sol' in split_name or 'Agar' in split_name:
            growth='Solid'
            if 'Agarose' in split_name:
                growthInd = split_name.index('Agarose')
            elif 'sol' in split_name:
                growthInd = split_name.index('sol')
            elif 'Agar' in split_name:
                growthInd = split_name.index('Agar')
        else:
            growth='Liquid'
            if 'liq' in split_name:
                growthInd = split_name.index('liq')
            else:
                growthInd = len(split_name)-3
        separator = '_'
        strain =  separator.join(split_name[0:growthInd])
        combined_csv.Strain.loc[(combined_csv['Label']==file)]=strain
        combined_csv.Growth.loc[(combined_csv['Label']==file)]=growth
combined_csv.to_csv( saved_file, index=False, encoding='utf-8-sig')
combined_csv.head()

,,Label,Area,Mean,Min,Max,X,Y,BiologicalReplicate,CellArea,...,MeanPoleBright,TotalFluorescencePoleBright,MinPoleBright,MaxPoleBright,StdPoleBright,Nb_Pili_PoleBright,Nb_Flagella_PoleBright,PolarRatio,Strain,Growth
0,1,mNG_pilH_fliC-_liq_Gasket_0_event10_tirf_RAW_S...,814,255,255,255,34.54668,33.15233,1,922,...,303.00641,47269.0,134,497,91.57077,5,0,0.89864,nan,nan
1,1,mNG_pilH_fliC-_liq_Gasket_0_event11_tirf_RAW_S...,856,255,255,255,43.07827,26.77804,1,969,...,247.28467,33878.0,112,446,89.25838,1,0,0.92396,nan,nan
2,1,mNG_pilH_fliC-_liq_Gasket_0_event12_tirf_RAW_S...,741,255,255,255,36.94399,27.26923,1,839,...,307.96350,42191.0,159,456,83.99198,1,0,0.90893,nan,nan
3,1,mNG_pilH_fliC-_liq_Gasket_0_event13_tirf_RAW_S...,769,255,255,255,45.72367,15.27243,1,858,...,299.14744,46667.0,100,457,105.57613,0,0,0.82365,nan,nan
4,2,mNG_pilH_fliC-_liq_Gasket_0_event13_tirf_RAW_S...,626,255,255,255,35.06070,23.45048,1,710,...,238.77372,32712.0,113,329,59.60001,1,0,0.88345,nan,nan


In [8]:
df = pd.read_csv(saved_file, sep=',', na_values='*')
df['PoleRange']=df['AreaPoleDim']/df['CellArea']
pole_range=df.PoleRange.mean()
print(pole_range)
ref_ratio = (2*pole_range)/(1-2*pole_range)
df['PercentTotalFluoDim']=df['TotalFluorescencePoleDim']/df['CellTotalFluorescence']
df['PercentTotalFluoBright']=df['TotalFluorescencePoleBright']/df['CellTotalFluorescence']
df['PercentTotalFluoCenter']=(df['CellTotalFluorescence']-df['TotalFluorescencePoleBright']-df['TotalFluorescencePoleDim'])/df['CellTotalFluorescence']

df['polar_ratio']=(df['TotalFluorescencePoleDim']+df['TotalFluorescencePoleBright'])/(df['CellTotalFluorescence']-(df['TotalFluorescencePoleDim']+df['TotalFluorescencePoleBright']))/((2*df['PoleRange']/(1-2*df['PoleRange'])))

df['TotalPili']=df['Nb_Pili_PoleDim']+df['Nb_Pili_PoleBright']
df['TotalFlagella']=df['Nb_Flagella_PoleDim']+df['Nb_Flagella_PoleBright']


y_param1='Nb_Pili_PoleDim'
y_param2='Nb_Pili_PoleBright'
tot_pili_limit=12
Ncells_per_pili_Dim=np.empty((tot_pili_limit), dtype='int16')
Ncells_per_pili_Bright=np.empty((tot_pili_limit), dtype='int16')
N=len(df)
print('Dim pole:')
print('NCells='+str(N), end=', ')
for i in range(tot_pili_limit):
    Ncells_per_pili_Dim[i]=len(df.loc[(df[y_param1]==i)])
    if (i<tot_pili_limit-1):
        print('N_'+str(i)+'pili='+str(Ncells_per_pili_Dim[i]), end=', ')
    else: print('N_'+str(i)+'pili='+str(Ncells_per_pili_Dim[i]))
print('Bright pole:')
print('Ncells='+str(N), end=', ')
for i in range(tot_pili_limit):
    Ncells_per_pili_Bright[i]=len(df.loc[(df[y_param2]==i)])
    if (i<tot_pili_limit-1):
        print('N_'+str(i)+'pili='+str(Ncells_per_pili_Bright[i]), end=', ')
    else: print('N_'+str(i)+'pili='+str(Ncells_per_pili_Bright[i]))
        
df.head()

0.15765268617712616
Dim pole:
NCells=97, N_0pili=50, N_1pili=14, N_2pili=14, N_3pili=6, N_4pili=6, N_5pili=4, N_6pili=2, N_7pili=0, N_8pili=1, N_9pili=0, N_10pili=0, N_11pili=0
Bright pole:
Ncells=97, N_0pili=38, N_1pili=13, N_2pili=9, N_3pili=10, N_4pili=10, N_5pili=6, N_6pili=5, N_7pili=1, N_8pili=3, N_9pili=1, N_10pili=1, N_11pili=0


,,Label,Area,Mean,Min,Max,X,Y,BiologicalReplicate,CellArea,...,PolarRatio,Strain,Growth,PoleRange,PercentTotalFluoDim,PercentTotalFluoBright,PercentTotalFluoCenter,polar_ratio,TotalPili,TotalFlagella
0,1,mNG_pilH_fliC-_liq_Gasket_0_event10_tirf_RAW_S...,814,255,255,255,34.54668,33.15233,1,922,...,0.89864,NaN,NaN,0.169197,0.151673,0.163223,0.685104,0.898641,5,0
1,1,mNG_pilH_fliC-_liq_Gasket_0_event11_tirf_RAW_S...,856,255,255,255,43.07827,26.77804,1,969,...,0.92396,NaN,NaN,0.141383,0.131351,0.135655,0.732994,0.923963,1,0
2,1,mNG_pilH_fliC-_liq_Gasket_0_event12_tirf_RAW_S...,741,255,255,255,36.94399,27.26923,1,839,...,0.90893,NaN,NaN,0.163290,0.148805,0.157132,0.694063,0.908932,1,0
3,1,mNG_pilH_fliC-_liq_Gasket_0_event13_tirf_RAW_S...,769,255,255,255,45.72367,15.27243,1,858,...,0.82365,NaN,NaN,0.181818,0.154466,0.165567,0.679968,0.823652,2,0
4,2,mNG_pilH_fliC-_liq_Gasket_0_event13_tirf_RAW_S...,626,255,255,255,35.06070,23.45048,1,710,...,0.88345,NaN,NaN,0.192958,0.174771,0.182224,0.643005,0.883454,1,0


In [9]:
sumPoles=(Ncells_per_pili_Dim+Ncells_per_pili_Bright)
Prob_Dim_if_nb_pili = [(Ncells_per_pili_Dim[i]/sumPoles[i]) for i in range(tot_pili_limit) if sumPoles[i]>0]
Prob_Bright_if_nb_pili=[(Ncells_per_pili_Bright[i]/sumPoles[i]) for i in range(tot_pili_limit) if sumPoles[i]>0]

Prob_per_pili_Dim= Ncells_per_pili_Dim/N
Prob_per_pili_Bright= Ncells_per_pili_Bright/N

In [10]:
p0 = bokeh.plotting.figure(
    width=600, 
    height=600, 
    x_axis_label='# pili', 
    y_axis_type='linear',
    y_axis_label ='P(# pili)',
    title="Probability of having # pili"
)

x_1=range(tot_pili_limit)


p0.line(
    x=x_1,
    y=Prob_per_pili_Dim, 
    line_color = 'blue',
    #fill_color = 'blue',
    alpha=0.7,
    #legend = labelsAll[i]
    legend = 'Dim Pole'
)


p0.line(
    x=x_1,
    y=Prob_per_pili_Bright, 
    line_color = 'red',
    #fill_color = 'red',
    alpha=0.7,
    #legend = labelsAll[i]
    legend = 'Bright pole'
)


bokeh.io.show(p0)

In [11]:
p1 = bokeh.plotting.figure(
    width=600, 
    height=600, 
    x_axis_label='# pili', 
    y_axis_type='linear',
    y_axis_label ='P(# pili|pole state)',
    title="Probability of having # pili if bright or dim pole"
)

x_1=range(len(Prob_Dim_if_nb_pili))


p1.line(
    x=x_1,
    y=Prob_Dim_if_nb_pili, 
    line_color = 'blue',
    #fill_color = 'blue',
    alpha=0.7,
    #legend = labelsAll[i]
    legend = 'Dim Pole'
)


p1.line(
    x=x_1,
    y=Prob_Bright_if_nb_pili, 
    line_color = 'red',
    #fill_color = 'red',
    alpha=0.7,
    #legend = labelsAll[i]
    legend = 'Bright pole'
)


bokeh.io.show(p1)

In [12]:
nb_pili=np.zeros((tot_pili_limit, 1))
boot_meanDim=np.zeros((tot_pili_limit, 1))
boot_medianDim=np.zeros((tot_pili_limit, 1))
boot_IC_meanDim=np.zeros((tot_pili_limit, 2))
boot_IC_medianDim=np.zeros((tot_pili_limit, 2))
boot_meanBright=np.zeros((tot_pili_limit, 1))
boot_medianBright=np.zeros((tot_pili_limit, 1))
boot_IC_meanBright=np.zeros((tot_pili_limit, 2))
boot_IC_medianBright=np.zeros((tot_pili_limit, 2))
for n_pili in range(tot_pili_limit):
    cell_fluorescence_arrayDim=df.TotalFluorescencePoleDim.loc[(df[y_param1]==n_pili)].values
    cell_fluorescence_arrayBright=df.TotalFluorescencePoleBright.loc[(df[y_param2]==n_pili)].values
    nb_pili[n_pili]=n_pili
    if (len(cell_fluorescence_arrayDim) > 0):
        bs_Dim_fluorescence_array=bootstrap_sampling(cell_fluorescence_arrayDim, len(cell_fluorescence_arrayDim), 1000)
        [bs_means, bs_medians, IC_means, IC_medians]=bootstrap_stats(bs_Dim_fluorescence_array)
        boot_meanDim[n_pili]=bs_means
        boot_IC_meanDim[n_pili,:]=IC_means
        boot_medianDim[n_pili]=bs_medians
        boot_IC_medianDim[n_pili,:]=IC_medians
    if (len(cell_fluorescence_arrayBright) > 0):
        bs_Bright_fluorescence_array=bootstrap_sampling(cell_fluorescence_arrayBright, len(cell_fluorescence_arrayBright), 1000)
        [bs_means, bs_medians, IC_means, IC_medians]=bootstrap_stats(bs_Bright_fluorescence_array)
        boot_meanBright[n_pili]=bs_means
        boot_IC_meanBright[n_pili,:]=IC_means
        boot_medianBright[n_pili]=bs_medians
        boot_IC_medianBright[n_pili,:]=IC_medians

names = ['TotalPili', 'bootMeanDim','bootMedianDim', 'bootMeanBright','bootMedianBright']
data = np.concatenate((nb_pili, boot_meanDim, boot_medianDim, boot_meanBright, boot_medianBright), axis=1)
df_boot_stats=pd.DataFrame(data=np.transpose(data), index=names).T
df_boot_stats['IC_meanDim']=list(boot_IC_meanDim)
df_boot_stats['IC_medianDim']=list(boot_IC_medianDim)
df_boot_stats['IC_meanBright']=list(boot_IC_meanBright)
df_boot_stats['IC_medianBright']=list(boot_IC_medianBright)
df_boot_stats

,TotalPili,bootMeanDim,bootMedianDim,bootMeanBright,bootMedianBright,IC_meanDim,IC_medianDim,IC_meanBright,IC_medianBright
0,0.0,30568.014840,30287.5,33833.550105,33458.5,"[27557.735, 33336.0815]","[27194.5, 33544.3125]","[29853.49407894737, 38093.89539473684]","[29232.0, 35735.0]"
1,1.0,34385.635786,34785.0,38711.277077,40546.0,"[28369.53214285714, 41468.69821428571]","[27466.0, 39488.0]","[34036.817307692305, 44250.0826923077]","[32712.0, 42191.0]"
2,2.0,39514.905643,38993.5,38900.745889,42068.0,"[34537.47678571429, 43969.33571428571]","[33754.5, 43004.0]","[30966.897222222222, 46275.549999999996]","[29206.0, 43841.0]"
3,3.0,39232.135500,39033.0,44983.587000,50948.0,"[33405.4875, 45163.549999999996]","[30454.5, 48106.5]","[32625.617500000004, 55401.9675]","[29315.5, 58494.0]"
4,4.0,50388.641333,46463.5,53681.158900,51846.0,"[44876.333333333336, 57804.25]","[43699.5, 61119.5]","[46115.549999999996, 60995.1375]","[44042.5, 66106.0]"
5,5.0,48801.654750,46837.5,50824.991333,50787.0,"[42101.0, 57176.5]","[40198.0, 60947.0]","[45133.666666666664, 56465.333333333336]","[42488.5, 59304.5]"
6,6.0,38562.787500,38362.5,45240.713400,44594.0,"[32640.0, 44085.0]","[32640.0, 44085.0]","[37956.8, 54075.4]","[33464.0, 61745.0]"
7,7.0,0.000000,0.0,61112.000000,61112.0,"[0.0, 0.0]","[0.0, 0.0]","[61112.0, 61112.0]","[61112.0, 61112.0]"
8,8.0,57963.000000,57963.0,53401.734333,54056.0,"[57963.0, 57963.0]","[57963.0, 57963.0]","[47869.0, 58589.0]","[47869.0, 58589.0]"
9,9.0,0.000000,0.0,40821.000000,40821.0,"[0.0, 0.0]","[0.0, 0.0]","[40821.0, 40821.0]","[40821.0, 40821.0]"


In [13]:
x_param1='TotalFluorescencePoleDim'#'PercentTotalFluoDim'
y_param1='Nb_Pili_PoleDim'
x_param2='TotalFluorescencePoleBright' #'PercentTotalFluoBright'
y_param2='Nb_Pili_PoleBright'
param2='TotalPili'
IC1='IC_medianDim'
IC2='IC_medianBright'
param1='bootMedianDim'
param3='bootMedianBright'
p2 = bokeh.plotting.figure(
    width=600, 
    height=600, 
    x_axis_label='# Pili', 
    y_axis_type='linear',
    y_axis_label ='Pole Total fluorescence',
    title="Pole fluorescence vs #Pili (bootstrap median, 95% CI)"
)

p2.circle(
    source=df,
    x=y_param1,
    y=x_param1, 
    line_color = 'blue',
    fill_color = 'blue',
    alpha=0.7,
    #legend = labelsAll[i]
    legend = 'Dim Pole'
)
p2.circle(
    source=df,
    x=y_param2,
    y=x_param2, 
    line_color = 'red',
    fill_color = 'red',
    alpha=0.7,
    #legend = labelsAll[i]
    legend = 'Bright Pole'
)

for n_pili in list(df_boot_stats[param2]):
    b = [n_pili,n_pili]
    ic=df_boot_stats.loc[(df_boot_stats[param2] == n_pili), [IC1]].values[0][0]
    p2.line(
        x = b,
        y = ic,
        color = 'black',
        alpha=0.7,
        line_width=3
    )
    
for n_pili in list(df_boot_stats['TotalPili']):
    b = [n_pili,n_pili]
    ic=df_boot_stats.loc[(df_boot_stats[param2] == n_pili), [IC2]].values[0][0]
    p2.line(
        x = b,
        y = ic,
        color = 'black',
        alpha=0.7,
        line_width=3
    )

p2.line(
    source = df_boot_stats.loc[:, [param2, param1]],
    x = param2,
    y = param1,
    line_color = 'blue',
    alpha=0.6
)


p2.circle(
    source = df_boot_stats.loc[:, [param2, param1]],
    x = param2,
    y = param1,
    line_color = 'black',
    fill_color = 'white',
    alpha=0.6,
    size=10
)

p2.line(
    source = df_boot_stats.loc[:, [param2, param3]],
    x = param2,
    y = param3,
    line_color = 'red',
    alpha=0.6
) 

p2.circle(
    source = df_boot_stats.loc[:, [param2, param3]],
    x = param2,
    y = param3,
    line_color = 'black',
    fill_color = 'white',
    alpha=0.6,
    size=10
)  

p2.output_backend = 'svg'

bokeh.io.show(p2)

In [14]:
x_param3='CellTotalFluorescence'
y_param3='polar_ratio'

print(x_param3+' vs '+y_param3+':')
[spearman_r, spearman_p]=stats.spearmanr(df[x_param3], df[y_param3])
print('Spearman correlation = '+str(spearman_r)+', p-value = '+ str(spearman_p))

[pearson_r, pearson_p]=stats.pearsonr(df[x_param3], df[y_param3])
print('Pearson correlation = '+str(pearson_r)+', p-value = '+str(pearson_p))

CellTotalFluorescence vs polar_ratio:
Spearman correlation = 0.4834972648853355, p-value = 5.260022986337116e-07
Pearson correlation = 0.49021045584541256, p-value = 3.457092217189231e-07


In [15]:
x_param3='CellTotalFluorescence'
y_param3='polar_ratio'

p1 = bokeh.plotting.figure(
    width=600, 
    height=600, 
    x_axis_label='Total fluorescence', 
    y_axis_type='linear',
    y_axis_label ='Polar Ratio',
    title="Polar Ratio vs Cell Total fluorescence"
)

p1.circle(
    source=df,
    x=x_param3,
    y=y_param3, 
    line_color = 'black',
    fill_color = 'gray',
    alpha=0.7,
    #legend = labelsAll[i]
)

bokeh.io.show(p1)

In [16]:
y_param1='Nb_Pili_PoleDim'
y_param2='Nb_Pili_PoleBright'
x_param1='TotalFluorescencePoleDim'
x_param2='TotalFluorescencePoleBright'
parameterX='TotalPili'
parameterY='PoleTotalFluorescence'
poles_pili_dim=list(df[y_param1])
poles_pili_bright=list(df[y_param2])
poles_pili=poles_pili_dim+poles_pili_bright
poles_pili=np.asarray(poles_pili).reshape(len(poles_pili),1)
poles_fluo_dim=list(df[x_param1])
poles_fluo_bright=list(df[x_param2])
poles_fluo=poles_fluo_dim+poles_fluo_bright
poles_fluo=np.asarray(poles_fluo).reshape(len(poles_fluo),1)

names = [parameterX, parameterY]
data = np.concatenate((poles_pili, poles_fluo), axis=1)
df_long=pd.DataFrame(data=np.transpose(data), index=names).T

In [17]:
x_param3=parameterX
y_param3=parameterY

print(x_param3+' vs '+y_param3+':')
[spearman_r, spearman_p]=stats.spearmanr(df_long[x_param3], df_long[y_param3])
print('Spearman correlation = '+str(spearman_r)+', p-value = '+ str(spearman_p))

[pearson_r, pearson_p]=stats.pearsonr(df_long[x_param3], df_long[y_param3])
print('Pearson correlation = '+str(pearson_r)+', p-value = '+str(pearson_p))

TotalPili vs PoleTotalFluorescence:
Spearman correlation = 0.5256054634941106, p-value = 3.57476131207281e-15
Pearson correlation = 0.4847224402909194, p-value = 7.96390211855239e-13


In [18]:
nb_pili=np.zeros((tot_pili_limit, 1))
boot_mean=np.zeros((tot_pili_limit, 1))
boot_median=np.zeros((tot_pili_limit, 1))
boot_IC_mean=np.zeros((tot_pili_limit, 2))
boot_IC_median=np.zeros((tot_pili_limit, 2))

for n_pili in range(tot_pili_limit):
    Pole_fluorescence_array=df_long.PoleTotalFluorescence.loc[(df_long[parameterX]==n_pili)].values
    nb_pili[n_pili]=n_pili
    if (len(Pole_fluorescence_array) > 0):
        bs_fluorescence_array=bootstrap_sampling(Pole_fluorescence_array, len(Pole_fluorescence_array), 1000)
        [bs_means, bs_medians, IC_means, IC_medians]=bootstrap_stats(bs_fluorescence_array)
        boot_mean[n_pili]=bs_means
        boot_IC_mean[n_pili,:]=IC_means
        boot_median[n_pili]=bs_medians
        boot_IC_median[n_pili,:]=IC_medians

names = ['TotalPili', 'bootMean','bootMedian']
data = np.concatenate((nb_pili, boot_mean, boot_median), axis=1)
df_boot_stats_tot=pd.DataFrame(data=np.transpose(data), index=names).T
df_boot_stats_tot['IC_mean']=list(boot_IC_mean)
df_boot_stats_tot['IC_median']=list(boot_IC_median)
df_boot_stats_tot

,TotalPili,bootMean,bootMedian,IC_mean,IC_median
0,0.0,31932.298466,32072.5,"[29628.115625, 34266.118465909094]","[28922.0, 34224.475]"
1,1.0,36438.310889,35347.0,"[32069.254629629628, 40897.694444444445]","[32712.0, 40546.0]"
2,2.0,39317.908913,40504.0,"[34947.73586956522, 43796.19673913043]","[33883.0, 43538.0]"
3,3.0,42983.507625,45887.0,"[35039.671875, 50610.00625]","[35372.0, 51730.0]"
4,4.0,52626.186000,49571.0,"[47472.6734375, 58777.48281249999]","[44904.0, 62318.0]"
5,5.0,49978.545800,49117.0,"[45005.105, 54859.35249999999]","[43733.5, 57650.0]"
6,6.0,43159.523000,44085.0,"[37100.42857142857, 49806.96071428571]","[33464.0, 46011.0]"
7,7.0,61112.000000,61112.0,"[61112.0, 61112.0]","[61112.0, 61112.0]"
8,8.0,54704.007750,56009.5,"[50392.5, 58276.0]","[47869.0, 58589.0]"
9,9.0,40821.000000,40821.0,"[40821.0, 40821.0]","[40821.0, 40821.0]"


In [19]:
param2='TotalPili'
IC1='IC_median'
param1='bootMedian'

p3 = bokeh.plotting.figure(
    width=600, 
    height=600, 
    x_axis_label='# pili', 
    y_axis_type='linear',
    y_axis_label ='pole fluorescence',
    title="Pole total fluorescence vs # pili (bootstrap median, 95% CI)"
)

p3.circle(
    source=df_long,
    x=parameterX,
    y=parameterY, 
    line_color = 'green',
    fill_color = 'white',
    alpha=0.4,
    #legend = labelsAll[i]
)

for n_pili in list(df_boot_stats_tot[param2]):
    b = [n_pili,n_pili]
    ic=df_boot_stats_tot.loc[(df_boot_stats_tot[param2] == n_pili), [IC1]].values[0][0]
    p3.line(
        x = b,
        y = ic,
        color = 'black',
        alpha=0.7,
        line_width=3
    )

p3.line(
    source = df_boot_stats_tot.loc[:, [param2, param1]],
    x = param2,
    y = param1,
    line_color = 'black',
    alpha=0.6
)


p3.circle(
    source = df_boot_stats_tot.loc[:, [param2, param1]],
    x = param2,
    y = param1,
    line_color = 'black',
    fill_color = 'white',
    alpha=0.6,
    size=10
)

p3.output_backend = 'webgl'

bokeh.io.show(p3)